Importing libraries and setting up connection

In [578]:
import pymongo
from pymongo import MongoClient

In [579]:
# !pip install geopy

In [580]:
import pandas as pd
from pandas import DataFrame

In [581]:
import operator
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
from geopy.distance import geodesic

In [582]:
#!pip install folium
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [583]:
# Server connection
client = MongoClient("localhost:27017")

In [584]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [585]:
# Show names of existing databases
client.list_database_names()

['Ironhack', 'admin', 'config', 'ironhacktest', 'local']

In [586]:
# Connection to the database
db = client.get_database("Ironhack")

In [587]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [588]:
db.list_collection_names()

['companies']

In [589]:
collection = db.get_collection("companies")

### Goal: Place the new company offices in the best place

#### Company details

- Sector - GAMING industry
- Employees - (85-90)

#### Requirements

- Nearby to other companies in the sector.
- Near successful tech startups that have raised at least 1 Million dollars.
- Near to Starbucks. 
- At least one airport in the city.
- Near  se to vegan restaurants.
- Entertainment sites around.
- Daycare centers

### Location


### 1. Exploring the MongoDB dataset to select location

#### Obtain all companies that have less than 100 employees and that have an Appraisal Amount greater than 1,000,000.

In [590]:
filtered = {'ipo': {'$exists': True },
          'ipo.valuation_amount': {'$exists': True, '$gte': 1000000},
          'number_of_employees': { '$gt': 90 },
          'offices.city':{"$ne": "",}}  
project = {'_id': 0,'name': 1, 'ipo.valuation_amount': 1, 'offices.city': 1, 'number_of_employees': 1 }
result = collection.find(filtered, project)

pd.DataFrame.from_records(result)

,name,number_of_employees,offices,ipo
0,Facebook,5299,"[{'city': 'Menlo Park'}, {'city': 'Dublin'}, {...",{'valuation_amount': 104000000000}
1,Twitter,1300,[{'city': 'San Francisco'}],{'valuation_amount': 18100000000}
2,Yelp,800,[{'city': 'San Francisco'}],{'valuation_amount': 1300000000}
3,LinkedIn,4800,[{'city': 'Mountain View'}],{'valuation_amount': 9310000000}
4,Brightcove,220,"[{'city': 'Cambridge'}, {'city': 'Tokyo'}, {'c...",{'valuation_amount': 290000000}
5,KIT digital,200,"[{'city': 'New York'}, {'city': 'Prague'}, {'c...",{'valuation_amount': 235000000}
6,OpenTable,550,[{'city': 'San Francisco'}],{'valuation_amount': 1050000000}
7,ChannelAdvisor,300,[{'city': 'Morrisville'}],{'valuation_amount': 287000000}
8,Zillow,600,[{'city': 'Seattle'}],{'valuation_amount': 2550000000}
9,Wix,400,"[{'city': 'Tel Aviv'}, {'city': 'San Francisco'}]",{'valuation_amount': 750000000}


### In the data frame it is observed that San Francisco is the most repeated location

#### Get all companies located in San Francisco that have at least 90 employees but fewer than 200. Excluding companies that have a value of None in the Offices / Latitude field.

In [591]:
filtered = {'offices.city': {'$eq': 'San Francisco'}, 
         'number_of_employees': { '$gt': 90,'$lt': 200 }, 
         'offices.latitude' : { '$ne': None }}
project = {"_id":0,"name":1,"offices.city": 1,"offices.latitude":1,"offices.longitude":1}
results = collection.find(filtered, project)

pd.DataFrame.from_records(results)

,name,offices
0,PowerReviews,"[{'city': 'San Francisco', 'latitude': 37.7795..."
1,hi5,"[{'city': 'San Francisco', 'latitude': 37.7886..."
2,Tagged,"[{'city': 'San Francisco', 'latitude': 37.7753..."
3,Blurb,"[{'city': 'San Francisco', 'latitude': 37.7929..."
4,The Climate Corporation,"[{'city': 'San Francisco', 'latitude': 37.7828..."
5,99designs,"[{'city': 'San Francisco', 'latitude': 37.7955..."
6,LiveRail,"[{'city': 'San Francisco', 'latitude': 37.7871..."
7,Wikimedia Foundation,"[{'city': 'San Francisco', 'latitude': 37.7678..."
8,SquareTrade,"[{'city': 'San Francisco', 'latitude': 37.7895..."
9,ServePath,"[{'city': 'San Francisco', 'latitude': 37.7898..."


### We choose 'hi5' as the first option

In [592]:
filtered = {'name' : 'hi5' }
project = {"_id":0,"offices.city": 1,"offices.latitude":1,"offices.longitude":1}
results = collection.find(filtered, project)

print(list(results))


[{'offices': [{'city': 'San Francisco', 'latitude': 37.788668, 'longitude': -122.400558}]}]


#### Get location

In [594]:
latitude = 37.788668
longitude = -122.400558

In [595]:
# Insert into a dataframe
location = [{'name': 'MyCompany', 'lat': latitude, 'lng': longitude, 'tag': 'my_location'}]
df_my_company = pd.DataFrame(location)
print(df_my_company)


        name        lat         lng          tag
0  MyCompany  37.788668 -122.400558  my_location


#### Visualization on  map

In [227]:
#prefix = fa lo ponemos cuando usemos los iconos de fontawesome.

map_1 = folium.Map(location = [latitude, longitude], zoom_start = 15)
office_loc = folium.Marker(location = [latitude, longitude], tooltip = "My Location")
icono = Icon(color = "orange",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "My Location")

In [84]:
location = [latitude, longitude]
marker_office = Marker(location = location, icon = icono)
marker_office.add_to(map_1)
map_1

In [23]:
# Exportar dataset
# map_1.save("ubicación_seleccionada.html")

### Study of nearby location requirements

####  2.1. Tech startups
Near successful tech startups that have raised at least 1 Million dollars

#### Get all the businesses in San Francisco, founded after the year 2000, that have between 90 and 150 employees and that have raised at least 1 Million dollars. Excluding companies that have a value of None in the Offices / Latitude field.

In [596]:
filt1 = {'offices.city': {'$eq': "San Francisco"},
         'number_of_employees': { '$gt': 90,'$lt': 150 },
         'total_money_raised': {'$exists': True, '$ne': '$0' },
         'founded_year': { '$gte': 2000 },
         'offices.latitude' : { '$ne': None }
        }
project1 = {"_id":0,"name":1,"offices.city": 1,"offices.latitude":1,"offices.longitude": 1, 'total_money_raised': 1, 'founded_year': 1}
results = collection.find(filt1, project1)

df_1M = pd.DataFrame.from_records(results)
df_1M

,name,founded_year,total_money_raised,offices
0,hi5,2003,$52M,"[{'city': 'San Francisco', 'latitude': 37.7886..."
1,99designs,2008,$35M,"[{'city': 'San Francisco', 'latitude': 37.7955..."
2,LiveRail,2008,$12M,"[{'city': 'San Francisco', 'latitude': 37.7871..."
3,Wikimedia Foundation,2003,$74M,"[{'city': 'San Francisco', 'latitude': 37.7678..."
4,GitHub,2008,$100M,"[{'city': 'San Francisco', 'latitude': 37.7751..."
5,Cloudmark,2001,$38.5M,"[{'city': 'San Francisco', 'latitude': 37.7795..."
6,Minted,2007,$52.7M,"[{'city': 'San Francisco', 'latitude': 37.7974..."
7,Twilio,2007,$104M,"[{'city': 'San Francisco', 'latitude': 37.7898..."
8,Canopy Financial,2004,$89.5M,"[{'city': 'San Francisco', 'latitude': 37.7913..."
9,ZoomSystems,2002,$87M,"[{'city': 'San Francisco', 'latitude': 37.7816..."


In [597]:
# Create new columns to indicate the coordinates and the tag.

def split_coordinates_lat(row):
    if 'offices' in row:
        for office in row['offices']:
            if office['city'] == 'San Francisco':
                return office['latitude']
    return None

def split_coordinates_lng(row):
    if 'offices' in row:
        for office in row['offices']:
            if office['city'] == 'San Francisco':
                return office['longitude']
    return None

def split_tag(row):
    if 'offices' in row:
        for office in row['offices']:
            if office['city'] == 'San Francisco':
                return 'Tech startups'
    return None

df_1M['lat'] = df_1M.apply(split_coordinates_lat, axis=1) 
df_1M['lng'] = df_1M.apply(split_coordinates_lng, axis=1) 
df_1M['tag'] = df_1M.apply(split_tag, axis=1)


# Delete all columns used for filter display
df_1M = df_1M.drop(['founded_year', 'total_money_raised', 'offices'], axis=1)
def_tech_startups = df_1M
def_tech_startups

,name,lat,lng,tag
0,hi5,37.788668,-122.400558,Tech startups
1,99designs,37.795531,-122.400598,Tech startups
2,LiveRail,37.787183,-122.397759,Tech startups
3,Wikimedia Foundation,37.767850,-122.392839,Tech startups
4,GitHub,37.775196,-122.419204,Tech startups
5,Cloudmark,37.779507,-122.390710,Tech startups
6,Minted,37.797435,-122.403175,Tech startups
7,Twilio,37.789850,-122.400683,Tech startups
8,Canopy Financial,37.791388,-122.392148,Tech startups
9,ZoomSystems,37.781689,-122.391061,Tech startups


In [610]:
# Create new column with distance between selected point and tag
def geo_distance(row):
    my_loc = (37.788668, -122.400558)
    row_loc = (row['lat'], row['lng'])
    return geodesic(my_loc, row_loc).miles

def_tech_startups['distance'] = def_tech_startups.apply(geo_distance, axis=1)
def_tech_startups = def_tech_startups.sort_values(['tag', 'distance'], ascending=[True, True])
def_tech_startups = def_tech_startups.reset_index(drop=True, inplace=False)
def_tech_startups 

,name,lat,lng,tag,distance
0,hi5,37.788668,-122.400558,Tech startups,0.000000
1,Twilio,37.789850,-122.400683,Tech startups,0.081800
2,LiveRail,37.787183,-122.397759,Tech startups,0.184280
3,99designs,37.795531,-122.400598,Tech startups,0.473309
4,Canopy Financial,37.791388,-122.392148,Tech startups,0.497053
5,Minted,37.797435,-122.403175,Tech startups,0.621371
6,ZoomSystems,37.781689,-122.391061,Tech startups,0.708440
7,Cloudmark,37.779507,-122.390710,Tech startups,0.830513
8,GitHub,37.775196,-122.419204,Tech startups,1.380213
9,Wikimedia Foundation,37.767850,-122.392839,Tech startups,1.496649


In [612]:
min_distance = def_tech_startups["distance"].min()


print("The closest is to:", min_distance, "miles")
closest_tech_startups = def_tech_startups.loc[1]
closest_tech_startups

The closest is to: 0.0 miles


name               Twilio
lat               37.7898
lng              -122.401
tag         Tech startups
distance        0.0817998
Name: 1, dtype: object

### 2. Connecting with Foursquare Api.

In [601]:
# Creating environment variables for identification in Foursquare
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [602]:
# Variable definition for location coordinates
office_location = {'type': 'Point', 'coordinates': [latitude, longitude]}
print (office_location)

{'type': 'Point', 'coordinates': [37.788668, -122.400558]}


#### 2.2. Starbucks
Executives like Starbucks A LOT

In [604]:
# Obtaining information from the Foursquare API, related to the Starbucks located within a radius of 500 m.
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [605]:
parameters = {"client_id" : f"{CLIENT_ID}",
              "client_secret" : f"{CLIENT_SECRET}",
              "v": "20210419",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":"starbucks",
              "radius":500}

In [606]:
resp = requests.get(url= url_query, params = parameters).json()
print(resp.keys())

data = resp.get("response")
# print(data)

dict_keys(['meta', 'response'])


In [607]:
sites = []
group = data['groups'][0]
for item in group['items']:
    site_dict = {}
    site_dict['name'] = item['venue']['name']
    site_dict['lat'] = item['venue']['location']['lat']
    site_dict['lng'] = item['venue']['location']['lng']
    site_dict['tag'] = 'starbucks'
    
    sites.append(site_dict)
    
# print(sites)

In [608]:
# Insert into a dataframe
df_starbucks = pd.DataFrame(sites)
df_starbucks

,name,lat,lng,tag
0,Starbucks,37.789651,-122.396622,starbucks
1,Starbucks,37.791939,-122.397402,starbucks
2,Starbucks,37.790110,-122.395610,starbucks
3,Starbucks,37.791104,-122.400951,starbucks
4,Starbucks,37.788792,-122.400710,starbucks
5,Starbucks,37.787360,-122.403180,starbucks
6,Starbucks,37.789462,-122.397582,starbucks
7,Starbucks,37.785782,-122.396687,starbucks
8,Starbucks,37.790425,-122.405773,starbucks
9,Starbucks,37.787902,-122.401144,starbucks


In [620]:
# Create new column with distance between selected point and tag

def geo_distance(row):
    my_loc = (37.788668, -122.400558)
    row_loc = (row['lat'], row['lng'])
    return geodesic(my_loc, row_loc).miles

df_starbucks['distance'] = df_starbucks.apply(geo_distance, axis=1)
df_starbucks = df_starbucks.sort_values(['tag', 'distance'], ascending=[True, True])
df_starbucks = df_starbucks.reset_index(drop=True, inplace=False)
df_starbucks.head()

,name,lat,lng,tag,distance
0,Starbucks,37.788792,-122.400710,starbucks,0.011923
1,Starbucks,37.787902,-122.401144,starbucks,0.061794
2,Starbucks,37.789594,-122.402032,starbucks,0.102894
3,Starbucks,37.791104,-122.400951,starbucks,0.169411
4,Starbucks,37.787360,-122.403180,starbucks,0.169508


In [615]:
min_distance = df_starbucks["distance"].min()


print("The closest is to:", min_distance, "miles")
closest_starbucks = df_starbucks.loc[0]
closest_starbucks

The closest is to: 0.011923223971605636 miles


name        Starbucks
lat           37.7888
lng          -122.401
tag         starbucks
distance    0.0119232
Name: 0, dtype: object

#### 2.3 Airport.
Account managers need to travel a lot

In [328]:
# Obtaining information from the Foursquare API, related to the Airport located within a radius of 230000 m.
url_query = 'https://api.foursquare.com/v2/venues/search'
airport = "4bf58dd8d48988d1ed931735"

In [329]:
parameters = {"client_id" : f"{CLIENT_ID}",
              "client_secret" : f"{CLIENT_SECRET}",
              "v": "20210419",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "categoryId": f"{airport}",
              "radius":23000}

# "query":f"{airport}"
# "query":"airport"

In [331]:
resp = requests.get(url= url_query, params = parameters).json()
print(resp.keys())
# print(json.dumps(resp, indent=4, sort_keys=True))
data = resp.get("response")
# print(data)

dict_keys(['meta', 'response'])


In [337]:
sites = []
# group = data['groups'][0]
for item in data['venues']:
    site_dict = {}
    site_dict['name'] = item['name']
    site_dict['lat'] = item['location']['lat']
    site_dict['lng'] = item['location']['lng']
    site_dict['tag'] = 'Airport'
    
    sites.append(site_dict)
    
# print(sites) 4bf58dd8d48988d1ed931735

# Insert into a dataframe
df = pd.DataFrame(sites)
df

,name,lat,lng,tag
0,Boarding Area D,37.617393,-122.381837,Airport
1,Terminal 3 Baggage Claim,37.618107,-122.386107,Airport
2,San Francisco International Airport (SFO) (San...,37.616713,-122.387094,Airport
3,Oakland International Airport (OAK) (Oakland I...,37.711936,-122.213416,Airport
4,The Centurion Lounge,37.617609,-122.387915,Airport
5,United Club,37.619309,-122.386880,Airport
6,Terminal 3,37.618217,-122.386354,Airport
7,Baggage Claim 1-2-3,37.617988,-122.387052,Airport
8,Terminal 2,37.616930,-122.383806,Airport
9,United Airlines Check-in,37.617950,-122.386998,Airport


In [422]:
# Function to keep the general rows
import re

def get_airport(row):

    pattern1 = r'.*\([A-Z]{3}\).*'
    sequence = row['name']

    if re.match(pattern1, sequence):
        return True
    
    return False

# Create column for filtering
df['Temporal'] = df.apply(get_airport, axis=1)
df

,name,lat,lng,tag,Temporal
0,Boarding Area D,37.617393,-122.381837,Airport,False
1,Terminal 3 Baggage Claim,37.618107,-122.386107,Airport,False
2,San Francisco International Airport (SFO) (San...,37.616713,-122.387094,Airport,True
3,Oakland International Airport (OAK) (Oakland I...,37.711936,-122.213416,Airport,True
4,The Centurion Lounge,37.617609,-122.387915,Airport,False
5,United Club,37.619309,-122.386880,Airport,False
6,Terminal 3,37.618217,-122.386354,Airport,False
7,Baggage Claim 1-2-3,37.617988,-122.387052,Airport,False
8,Terminal 2,37.616930,-122.383806,Airport,False
9,United Airlines Check-in,37.617950,-122.386998,Airport,False


In [423]:
# Delete the rows whose value is False in the column 'Temporary'
df_airport = df.copy()
df_airport.drop(df_airport[df_airport['Temporal'] == False].index, inplace = True)
df_airport

,name,lat,lng,tag,Temporal
2,San Francisco International Airport (SFO) (San...,37.616713,-122.387094,Airport,True
3,Oakland International Airport (OAK) (Oakland I...,37.711936,-122.213416,Airport,True


In [424]:
# Delete all columns used for filter display 'Temporal'
df_airport = df_airport.drop(['Temporal'], axis=1)
df_airport = df_airport.reset_index(drop=True, inplace=False)
df_airport

,name,lat,lng,tag
0,San Francisco International Airport (SFO) (San...,37.616713,-122.387094,Airport
1,Oakland International Airport (OAK) (Oakland I...,37.711936,-122.213416,Airport


#### 2.4 Nightlife Spot
Everyone in the company is between 25 and 40

In [374]:
# Obtaining information from the Foursquare API, related to the Night clubs located within a radius of 500 m.
url_query = 'https://api.foursquare.com/v2/venues/search'
nightlife_spot = "4d4b7105d754a06376d81259"

In [375]:
parameters = {"client_id" : f"{CLIENT_ID}",
              "client_secret" : f"{CLIENT_SECRET}",
              "v": "20210419",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "categoryId": f"{nightlife_spot}",
              "radius":500}

In [376]:
resp = requests.get(url= url_query, params = parameters).json()
print(resp.keys())

data = resp.get("response")
# print(data)

dict_keys(['meta', 'response'])


In [377]:
sites = []
for item in data['venues']:
    site_dict = {}
    site_dict['name'] = item['name']
    site_dict['lat'] = item['location']['lat']
    site_dict['lng'] = item['location']['lng']
    site_dict['tag'] = 'Nightlife Spot'
    
    sites.append(site_dict)
    
# print(sites)
df_nightlife_spot = pd.DataFrame(sites)
df_nightlife_spot

,name,lat,lng,tag
0,Sutter Station,37.789960,-122.400915,Nightlife Spot
1,Novela,37.786927,-122.401246,Nightlife Spot
2,Galleria Park Hotel Lobby,37.789696,-122.403446,Nightlife Spot
3,Bluestone Lane,37.787813,-122.402913,Nightlife Spot
4,Kona’s Street Market Bar,37.787040,-122.403241,Nightlife Spot
5,Kona's Street Market,37.787064,-122.403270,Nightlife Spot
6,Sam’s Tavern,37.790932,-122.403559,Nightlife Spot
7,ALX by Alexander's Steakhouse,37.784679,-122.398408,Nightlife Spot
8,Natoma Cabana,37.787600,-122.398502,Nightlife Spot
9,The Pink Elephant,37.786813,-122.400456,Nightlife Spot


In [619]:
# Create new column with distance between selected point and tag

def geo_distance(row):
    my_loc = (37.788668, -122.400558)
    row_loc = (row['lat'], row['lng'])
    return geodesic(my_loc, row_loc).miles

df_nightlife_spot['distance'] = df_nightlife_spot.apply(geo_distance, axis=1)
df_nightlife_spot = df_nightlife_spot.sort_values(['tag', 'distance'], ascending=[True, True])
df_nightlife_spot = df_nightlife_spot.reset_index(drop=True, inplace=False)
df_nightlife_spot.head() 

,name,lat,lng,tag,distance
0,Atlas Tap Room,37.787880,-122.400027,Nightlife Spot,0.061661
1,Sutter Station,37.789960,-122.400915,Nightlife Spot,0.091218
2,Novela,37.786927,-122.401246,Nightlife Spot,0.125852
3,The Pink Elephant,37.786813,-122.400456,Nightlife Spot,0.128047
4,Natoma Cabana,37.787600,-122.398502,Nightlife Spot,0.134470


In [618]:
min_distance = df_nightlife_spot["distance"].min()


print("The closest is to:", min_distance, "miles")
closest_nightlife_spot = df_nightlife_spot.loc[0]
closest_nightlife_spot

The closest is to: 0.06166116188982341 miles


name        Atlas Tap Room
lat                37.7879
lng                 -122.4
tag         Nightlife Spot
distance         0.0616612
Name: 0, dtype: object

#### 2.5 Vegan restaurants.
The CEO is vegan.

In [392]:
# Obtaining information from the Foursquare API, related to the Vegan restaurants located within a radius of 500 m.
url_query = 'https://api.foursquare.com/v2/venues/explore'
vegan_restaurants = "4bf58dd8d48988d1d3941735"

In [393]:
parameters = {"client_id" : f"{CLIENT_ID}",
              "client_secret" : f"{CLIENT_SECRET}",
              "v": "20210419",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "categoryId": f"{vegan_restaurants}",
              "radius":500}

In [394]:
resp = requests.get(url= url_query, params = parameters).json()
print(resp.keys())

data = resp.get("response")
# print(data)

dict_keys(['meta', 'response'])


In [395]:
sites = []
group = data['groups'][0]
for item in group['items']:
    site_dict = {}
    site_dict['name'] = item['venue']['name']
    site_dict['lat'] = item['venue']['location']['lat']
    site_dict['lng'] = item['venue']['location']['lng']
    site_dict['tag'] = 'Vegan restaurants'
    
    sites.append(site_dict)
    
# print(sites)
df_vegan = pd.DataFrame(sites)
df_vegan

,name,lat,lng,tag
0,sweetgreen,37.787262,-122.398635,Vegan restaurants
1,Mixt,37.788844,-122.399149,Vegan restaurants
2,Mixt,37.785751,-122.404124,Vegan restaurants
3,The Plant Cafe Organic,37.792525,-122.398465,Vegan restaurants
4,Elixiria,37.792053,-122.396844,Vegan restaurants
5,Mixt,37.791589,-122.400787,Vegan restaurants
6,Sunrise Deli,37.788611,-122.400532,Vegan restaurants
7,The Plant Cafe Organic,37.791888,-122.402499,Vegan restaurants
8,Amber,37.785967,-122.404567,Vegan restaurants
9,Fang,37.785667,-122.399703,Vegan restaurants


In [621]:
# Create new column with distance between selected point and tag

def geo_distance(row):
    my_loc = (37.788668, -122.400558)
    row_loc = (row['lat'], row['lng'])
    return geodesic(my_loc, row_loc).miles

df_vegan['distance'] = df_vegan.apply(geo_distance, axis=1)
df_vegan = df_vegan.sort_values(['tag', 'distance'], ascending=[True, True])
df_vegan = df_vegan.reset_index(drop=True, inplace=False)
df_vegan.head() 

,name,lat,lng,tag,distance
0,Sunrise Deli,37.788611,-122.400532,Vegan restaurants,0.004189
1,Dosa Brothers,37.789021,-122.401848,Vegan restaurants,0.074679
2,Mixt,37.788844,-122.399149,Vegan restaurants,0.078068
3,Ginto Izakaya Japonaise,37.788313,-122.402916,Vegan restaurants,0.131362
4,sweetgreen,37.787262,-122.398635,Vegan restaurants,0.143104


In [622]:
min_distance = df_vegan["distance"].min()


print("The closest is to:", min_distance, "miles")
closest_vegan = df_vegan.loc[0]
closest_vegan

The closest is to: 0.004189114153864005 miles


name             Sunrise Deli
lat                   37.7886
lng                  -122.401
tag         Vegan restaurants
distance           0.00418911
Name: 0, dtype: object

#### 2.6 Daycare.
30% of the company staff have at least 1 child.

In [383]:
# Obtaining information from the Foursquare API, related to the Daycare located within a radius of 500 m.
url_query = 'https://api.foursquare.com/v2/venues/explore'
daycare = "5744ccdfe4b0c0459246b4c7"

In [384]:
parameters = {"client_id" : f"{CLIENT_ID}",
              "client_secret" : f"{CLIENT_SECRET}",
              "v": "20210419",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "categoryId": f"{daycare}",
              "radius":500}

In [385]:
resp = requests.get(url= url_query, params = parameters).json()
print(resp.keys())

data = resp.get("response")
# print(data)

dict_keys(['meta', 'response'])


In [387]:
sites = []
group = data['groups'][0]
for item in group['items']:
    site_dict = {}
    site_dict['name'] = item['venue']['name']
    site_dict['lat'] = item['venue']['location']['lat']
    site_dict['lng'] = item['venue']['location']['lng']
    site_dict['tag'] = 'Daycare'
    
    sites.append(site_dict)
    
# print(sites)
df_daycare = pd.DataFrame(sites)
df_daycare

,name,lat,lng,tag
0,Kids by the Bay - Financial District,37.785471,-122.398037,Daycare
1,Bright Horizons,37.792097,-122.403744,Daycare
2,Marin Day Schools Fremont Street,37.789582,-122.395118,Daycare


In [623]:
# Create new column with distance between selected point and tag

def geo_distance(row):
    my_loc = (37.788668, -122.400558)
    row_loc = (row['lat'], row['lng'])
    return geodesic(my_loc, row_loc).miles

df_daycare['distance'] = df_daycare.apply(geo_distance, axis=1)
df_daycare = df_daycare.sort_values(['tag', 'distance'], ascending=[True, True])
df_daycare = df_daycare.reset_index(drop=True, inplace=False)
df_daycare.head() 

,name,lat,lng,tag,distance
0,Kids by the Bay - Financial District,37.785471,-122.398037,Daycare,0.260106
1,Bright Horizons,37.792097,-122.403744,Daycare,0.293826
2,Marin Day Schools Fremont Street,37.789582,-122.395118,Daycare,0.304370


In [624]:
min_distance = df_daycare["distance"].min()


print("The closest is to:", min_distance, "miles")
closest_daycare = df_daycare.loc[0]
closest_daycare

The closest is to: 0.2601060315122544 miles


name        Kids by the Bay - Financial District
lat                                      37.7855
lng                                     -122.398
tag                                      Daycare
distance                                0.260106
Name: 0, dtype: object

### 3. General dataset with all points located

In [549]:
# Concatenation of all datasets

df_all = pd.concat([df_my_company, def_tech_startups, df_starbucks, df_airport, df_nightlife_spot, df_vegan, df_daycare])
df_all = df_all.reset_index(drop=True, inplace=False)

df_all

,name,lat,lng,tag
0,MyCompany,37.788668,-122.400558,my_location
1,hi5,37.788668,-122.400558,Tech startups
2,99designs,37.795531,-122.400598,Tech startups
3,LiveRail,37.787183,-122.397759,Tech startups
4,Wikimedia Foundation,37.767850,-122.392839,Tech startups
...,...,...,...,...
71,Dosa Brothers,37.789021,-122.401848,Vegan restaurants
72,Ginto Izakaya Japonaise,37.788313,-122.402916,Vegan restaurants
73,Kids by the Bay - Financial District,37.785471,-122.398037,Daycare
74,Bright Horizons,37.792097,-122.403744,Daycare


In [550]:
df_all['tag'].value_counts()

Nightlife Spot       30
starbucks            16
Vegan restaurants    14
Tech startups        10
Daycare               3
Airport               2
my_location           1
Name: tag, dtype: int64

In [551]:
# Create new column with distance between selected point and tag

from geopy.distance import geodesic
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(geodesic(newport_ri, cleveland_oh).miles)

def geo_distance(row):
    my_loc = (37.788668, -122.400558)
    row_loc = (row['lat'], row['lng'])
    return geodesic(my_loc, row_loc).miles

df_all['distance'] = df_all.apply(geo_distance, axis=1)
df_all = df_all.sort_values(['tag', 'distance'], ascending=[True, True])
df_all = df_all.reset_index(drop=True, inplace=False)
df_all        

538.3904453677205


,name,lat,lng,tag,distance
0,Oakland International Airport (OAK) (Oakland I...,37.711936,-122.213416,Airport,11.533783
1,San Francisco International Airport (SFO) (San...,37.616713,-122.387094,Airport,11.882063
2,Kids by the Bay - Financial District,37.785471,-122.398037,Daycare,0.260106
3,Bright Horizons,37.792097,-122.403744,Daycare,0.293826
4,Marin Day Schools Fremont Street,37.789582,-122.395118,Daycare,0.304370
...,...,...,...,...,...
71,Starbucks,37.790110,-122.395610,starbucks,0.288499
72,Starbucks,37.785782,-122.396687,starbucks,0.290696
73,Starbucks,37.792190,-122.403540,starbucks,0.292642
74,Starbucks,37.793060,-122.399760,starbucks,0.306038


In [552]:
# Exportar dataset 1
# df_all.to_csv(r"C:\Users\sussy\CODE\Week_4\w4_geospatia_project\output\all_points_located.csv", index = False)

### 4. Visualization on  map

#### Heat map

In [446]:
heat_map = folium.Map (location = [latitude, longitude], zoom_start = 15)

In [447]:
tag_tech_startups = df_all[df_all.tag == "Tech startups"]
tag_tech_group = folium.FeatureGroup(name = "Tech startups")
HeatMap(data = tag_tech_startups[["lat","lng"]], radius = 15).add_to(tag_tech_group)
tag_tech_group.add_to(heat_map)

tag_starbucks = df_all[df_all.tag == "starbucks"]
starbucks_group = folium.FeatureGroup(name = "Starbucks")
HeatMap(data = tag_starbucks[["lat","lng"]], radius = 15).add_to(starbucks_group)
starbucks_group.add_to(heat_map)

tag_airport = df_all[df_all.tag == "Airport"]
airport_group = folium.FeatureGroup(name = "Airport")
HeatMap(data = tag_airport[["lat","lng"]], radius = 15).add_to(airport_group)
airport_group.add_to(heat_map)

tag_nightlife_spot = df_all[df_all.tag == "Nightlife Spot"]
nightlife_spot_group = folium.FeatureGroup(name = "Nightlife Spot")
HeatMap(data = tag_nightlife_spot[["lat","lng"]], radius = 15).add_to(nightlife_spot_group)
nightlife_spot_group.add_to(heat_map)

tag_vegan_restaurants = df_all[df_all.tag == "Vegan restaurants"]
vegan_restaurants_group = folium.FeatureGroup(name = "Vegan restaurants")
HeatMap(data = tag_vegan_restaurants[["lat","lng"]], radius = 15).add_to(vegan_restaurants_group)
vegan_restaurants_group.add_to(heat_map)

tag_daycare = df_all[df_all.tag == "Daycare"]
daycare_group = folium.FeatureGroup(name = "Daycare")
HeatMap(data = tag_daycare[["lat","lng"]], radius = 15).add_to(daycare_group)
daycare_group.add_to(heat_map)

In [448]:
icon = Icon(color = "white",
            prefix = "fa",
            icon = "building-o",
            icon_color = "green")
loc = {"location":[latitude, longitude],
       "tooltip": "My location"}

marker_office = Marker(**loc, icon = icono)
marker_office.add_to(heat_map)

In [449]:
folium.LayerControl(collapsed=False).add_to(heat_map)

In [450]:
heat_map

In [451]:
# Exportar dataset
# heat_map.save("all_heat_map.html")

#### Markers map 

In [496]:
map_all = folium.Map(location = [latitude, longitude], zoom_start = 15)

for i,row in df_all.iterrows():
    name = {"location" : [row["lat"], row["lng"]],
            "tooltip" : row["name"]}
    
    if row["tag"] == "starbucks":
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "white")  
        
    elif row["tag"] == 'Tech startups':
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "line-chart",
                    icon_color = "beige")      
        
    elif row["tag"] == 'my_location':
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "hand-spock-o",
                    icon_color = "beige")
        
    elif row["tag"] == 'Airport':
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "plane",
                    icon_color = "beige")
        
    elif row["tag"] == 'Nightlife Spot':
        icon = Icon(color = "darkpurple",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "beige")
        
    elif row["tag"] == 'Vegan restaurants':
        icon = Icon(color = "purple",
                    prefix = "fa",
                    icon = "lemon-o",
                    icon_color = "beige")
        
    elif row["tag"] == 'Daycare':
        icon = Icon(color = "pink",
                    prefix = "fa",
                    icon = "child",
                    icon_color = "beige")
        
        
    else:
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "building-o",
                    icon_color = "black")
        
    Marker(**name, icon = icon).add_to(map_all)
    
map_all

In [498]:
# Exportar dataset
# map_all.save("all_markers_map.html")